In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random as rd
from datetime import datetime, timedelta

import math
import gurobipy as gp
from gurobipy import GRB

In [2]:
now = datetime.now()
num_user = 50

# 출발 희망 시각[변환 전]
start = [ rd.randrange(60*17, 60*19, 5) for i in range(num_user)]
# 출발 희망 시각[변환 후]
start_time = [ datetime(now.year, now.month, now.day, hour=timedelta(minutes=a).seconds // 3600, minute=(timedelta(minutes=a).seconds // 60) % 60).strftime('%Y-%m-%d %H:%M:%S') for a in start]

# 대기 가능 시간
wait = [ rd.randrange(10, 60,10) for i in range(num_user)]

# 최대 대기 시각[변환 전]
tolerance = [start[i] + wait[i] for i in range(num_user)]
# 최대 대기기 시각[변환 후]
tolerance_time = [ datetime(now.year, now.month, now.day, hour=timedelta(minutes=a).seconds // 3600, minute=(timedelta(minutes=a).seconds // 60) % 60).strftime('%Y-%m-%d %H:%M:%S') for a in tolerance]

# 우선순위
priority = [ i for i in range(1,num_user+1)]

# range_time
range_time = [(start[i], tolerance[i]) for i in range(num_user)]

# ID
ID = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','AA','BB','CC','DD','EE','FF','GG','HH','II','JJ','KK','LL','MM','NN','OO','PP','QQ','RR','SS','TT','UU','VV','WW','XX','YY','ZZ']
id = ID[:num_user]

# 출발지 lat(위도)&lon(경도) [0,1000 사이의 난수] '도'가 아니라 가시를 위해 '초'로 표시하였다.
# start_lat = [round(1000*rd.random(),4) for i in range(num_user)]
# start_lon = [round(1000*rd.random(),4) for i in range(num_user)]
start_lat = [rd.randrange(100, 400, 100) for i in range(num_user)]
start_lon = [rd.randrange(100, 400, 100) for i in range(num_user)]
start_loc = [(start_lat[i], start_lon[i]) for i in range(num_user)]

# 출발지 lat(위도)&lon(경도) [0,1000 사이의 난수]
des_lat = [round(1000*rd.random(),4) for i in range(num_user)]
des_lon = [round(1000*rd.random(),4) for i in range(num_user)]
des_loc = [(des_lat[i], des_lon[i]) for i in range(num_user)]

In [4]:
df1 = pd.DataFrame(dict(zip(['ID','출발희망시각','최대대기시각','우선순위','시간범위','출발위치','도착위치'],[id, start_time, tolerance_time, priority, range_time, start_loc, des_loc])))
# df1 = df1.sort_values(by=['출발희망시각','최대대기시각']) # sorting
# df1 = df1.set_index('우선순위')
df1.head()

,ID,출발희망시각,최대대기시각,우선순위,시간범위,출발위치,도착위치
0,A,2023-04-15 18:55:00,2023-04-15 19:15:00,1,"(1135, 1155)","(200, 300)","(595.8121, 246.5783)"
1,B,2023-04-15 17:55:00,2023-04-15 18:15:00,2,"(1075, 1095)","(200, 200)","(58.0101, 563.9948)"
2,C,2023-04-15 18:55:00,2023-04-15 19:15:00,3,"(1135, 1155)","(300, 200)","(413.5576, 334.532)"
3,D,2023-04-15 17:50:00,2023-04-15 18:20:00,4,"(1070, 1100)","(300, 200)","(48.0403, 207.5821)"
4,E,2023-04-15 18:30:00,2023-04-15 18:50:00,5,"(1110, 1130)","(200, 300)","(542.1523, 466.5186)"


In [21]:
m = 5 # 처음 범위

while(True):
    n = 3 # 추가적으로 늘어나는 범위

    #########################################################################
    ################################# STEP1 #################################

    df2 = df1.copy()
    target_index = 1
    target_info = df2.loc[target_index]

    start, end = df2.loc[target_index]['시간범위']
    for index, row in df2.iterrows():
      st, en = df2.loc[index]['시간범위']
      if start>=en or end<=st:
        df2 = df2.drop(index, axis=0)

    #########################################################################
    ################################# STEP2 #################################

    # A의 위치정보
    start_lat, start_lon = df2.loc[target_index]['출발위치']
    end_lat, end_lon = df2.loc[target_index]['도착위치']

    print(f'range : {m/37:.4f}km')
    print()
    m += n # 5초가 default
    m = m*1000/37 # km를 초로 치환, 1초는 37m이다

    # 직선의 방정식
    def line_eq(x1, y1, x2, y2):
      m = (y2 - y1) / (x2 - x1)
      b = y1 - m * x1
      A, B, C = -m, 1, -b
      return A, B, C

    # 직선과 A 사이의 거리
    def distance(x1, y1, a, b, c):
      dist = abs(a*x1 + b*y1 + c) / math.sqrt(a*a + b*b)
      return dist

    for index, row in df2.iterrows():
      st_lat, st_lon = df2.loc[index]['출발위치']
      en_lat, en_lon = df2.loc[index]['도착위치']

      # 각 ID의 벡터
      a, b, c = line_eq(st_lat, st_lon, en_lat, en_lon)
      # 각 ID의 벡터와 A 출발점 사이의 거리
      dist_from_st = distance(start_lat, start_lon, a, b, c)
      # 각 ID의 벡터와 B 출발점 사이의 거리
      dist_from_en = distance(end_lat, end_lon, a, b, c)

      # print(dist_from_st, dist_from_en)

      if dist_from_st >= m and dist_from_en >= m:
        df2 = df2.drop(index, axis=0)

    # 조건을 만족하는 후보자가 없으면 거리를 늘린다. 자기자신 밖에 없는 경우 ==> 1
    if len(df2) == 1:
        print('need to increase range_BY STEP2')
        pass

    #########################################################################
    ################################# STEP3 #################################

    # A의 위치정보 => 코사인 유사도
    start_lat, start_lon = df2.loc[target_index]['출발위치']
    end_lat, end_lon = df2.loc[target_index]['도착위치']
    a_vec = np.array([start_lat-end_lat, start_lon-end_lon])
    # print(f'타깃 동승자의 vector: {a_vec}')

    def get_angle(v1, v2):
        """두 벡터 사이각을 계산하는 함수"""
        cos_sim = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

        if np.linalg.norm(v1) == 0 or np.linalg.norm(v2) == 0:
            return np.nan  # 벡터의 크기가 0이면 NaN 반환

        if np.abs(cos_sim) > 1:
            return np.nan  # 코사인 유사도가 1보다 크거나 -1보다 작으면 NaN 반환

        angle = np.arccos(cos_sim)
        return np.degrees(angle)  # 라디안을 도로 변환하여 반환

    for index, row in df2.iterrows():
      st_lat, st_lon = df2.loc[index]['출발위치']
      en_lat, en_lon = df2.loc[index]['도착위치']
      i_vec = np.array([st_lat-en_lat, st_lon-en_lon])

      if get_angle(a_vec, i_vec) > 90:
        df2 = df2.drop(index, axis=0)
      # else:
        # print(get_angle(a_vec, i_vec))

    # 조건을 만족하는 후보자가 없으면 거리를 늘린다.
    if len(df2) == 1:
        print('need to increase range_BY STEP3')
        pass

    #########################################################################
    ################################# STEP4 #################################

    # 자기자신과 매칭 방지를 위해 제거
    df2 = df2.drop(target_index, axis=0)
    epoch = 4

    def distance(x1, y1, x2, y2):
        dx = x2 - x1
        dy = y2 - y1
        dist = math.sqrt(dx*dx + dy*dy)
        return dist

    for index, row in df2.iterrows():
      st_lat, st_lon = start_lat, start_lon
      en_lat, en_lon = df2.loc[index]['도착위치']
      # lat_lon = {
      #     start_lat: start_lon,
      #     end_lat: end_lon,
      #     st_lat: st_lon,
      #     en_lat: en_lon
      #     }
      lat_lon = {
        'a_start': df2.loc[index]['출발위치'],
        'a_end': df2.loc[index]['도착위치'],
        'i_start': (start_lat, start_lon),
        'i_end': (end_lat, end_lon)
          }
      # 원점에서 떨어진 거리순으로 정렬
      dis_lat_lon = {
          distance(0, 0, start_lat, start_lon): 'a_start', # a 시작지점
          distance(0, 0, end_lat, end_lon): 'a_end', # a 끝지점
          distance(0, 0, st_lat, st_lon): 'i_start', # i 시작지점
          distance(0, 0, en_lat, en_lon): 'i_end' # i 끝지점
          }


      dis = sorted([distance(0, 0, start_lat, start_lon), distance(0, 0, end_lat,end_lon), distance(0, 0, st_lat, st_lon), distance(0, 0, en_lat, en_lon)])
      # print(dis)
      routing_lat_lon = [ dis_lat_lon[key] for key in dis]
      # print(routing_lat_lon) # 승하차순서
      routing_location = [ lat_lon[key] for key in routing_lat_lon]
      # print(routing_location) # 이동경로

      routing_lat = [lat for lat,lon in [ lat_lon[key] for key in routing_lat_lon]]
      routing_lon = [lon for lat,lon in [ lat_lon[key] for key in routing_lat_lon]]

      # print('routing_lat: ',routing_lat)
      # print('routing_lon: ',routing_lon)
      # print(f'A start: {start_lat, start_lon} A end: {end_lat, end_lon}')
      # print(f'I start: {st_lat, st_lon} I end: {en_lat, en_lon}')

      # 경유 node
      if routing_lat.index(start_lat) > routing_lat.index(end_lat) or routing_lat.index(st_lat) > routing_lat.index(en_lat):
        routing_lat = routing_lat[::-1]
        routing_lon = routing_lon[::-1]
        a_routing_idx = [routing_lat.index(start_lat), routing_lat.index(end_lat)]
        i_routing_idx = [routing_lat.index(st_lat), routing_lat.index(en_lat)]


      else:
        a_routing_idx = [routing_lat.index(start_lat), routing_lat.index(end_lat)]
        i_routing_idx = [routing_lat.index(st_lat), routing_lat.index(en_lat)]


      a_route_dist = 0
      a_dist = 0
      start_point = a_routing_idx[0]
      end_point = a_routing_idx[1]
      st_point = i_routing_idx[0]
      en_point = i_routing_idx[1]

      # print(start_point<end_point)


      # 0>1 , 2>3 인 경우 제외
      if (start_point, end_point) == (0,1) or (start_point, end_point) == (2,3):
        df2 = df2.drop(index, axis=0)
        # print('wrong match : tp1')
        # print()
      elif start_point > end_point or st_point > en_point:
        df2 = df2.drop(index, axis=0)
        # print('wrong match : tp2')
        # print()

      else:
        # print(start_point, end_point)

        # 타킷경유시간(분)
        point = start_point
        try:
          for i in range(epoch):
            # a가 노드
            x1 = routing_lat[point]
            y1 = routing_lon[point]
            # 다음 노드
            point += 1

            x2 = routing_lat[point]
            y2 = routing_lon[point]
            dist_btn_node = distance(x1, y1, x2, y2) * 0.037 # 초(각도) >> km(거리)
            a_route_dist += dist_btn_node
            if point==end_point:
              break
        except:
          pass

        a_route_time = 60/23 * a_route_dist # km(거리) >> 분(시간)
        df2.loc[index,'타깃경유시간(분)'] = a_route_time
        df2.loc[index,'타깃경유거리(km)'] = a_route_dist

        # 타킷원래시간(분)
        # a 출발
        x1 = routing_lat[start_point]
        y1 = routing_lon[start_point]
        # a 도착
        x2 = routing_lat[end_point]
        y2 = routing_lon[end_point]
        a_dist = distance(x1, y1, x2, y2) * 0.037
        a_time = 60/23 * a_dist
        df2.loc[index,'타깃원래시간(분)'] = a_time
        df2.loc[index,'타깃원래거리(km)'] = a_dist


        i_route_dist = 0
        i_dist = 0
        start_point = i_routing_idx[0]
        end_point = i_routing_idx[1]
        # print(start_point, end_point)

        # 경유시간(분)
        point = start_point
        try:
          for i in range(epoch):
            # a가 노드
            x1 = routing_lat[point]
            y1 = routing_lon[point]
            # 다음 노드
            point += 1
            x2 = routing_lat[point]
            y2 = routing_lon[point]
            dist_btn_node = distance(x1, y1, x2, y2) * 0.037 # 초(각도) >> km(거리)
            i_route_dist += dist_btn_node
            if point==end_point:
              break
        except:
          pass

        i_route_time = 60/23 * i_route_dist # km(거리) >> 분(시간)
        df2.loc[index,'경유시간(분)'] = i_route_time
        df2.loc[index,'경유거리(km)'] = i_route_dist

        # 원래시간(분)
        # i 출발
        x1 = routing_lat[start_point]
        y1 = routing_lon[start_point]
        # i 도착
        x2 = routing_lat[end_point]
        y2 = routing_lon[end_point]
        i_dist = distance(x1, y1, x2, y2) * 0.037
        i_time = 60/23 * i_dist
        df2.loc[index,'원래시간(분)'] = i_time
        df2.loc[index,'원래거리(km)'] = i_dist

        # print(routing_lat, routing_lon)
        # print()


        total_route_dist = 0
        point = 0

        # 전체체경유시간(분)
        try:
          for i in range(epoch):
            # a가 노드
            x1 = routing_lat[point]
            y1 = routing_lon[point]
            # 다음 노드
            point += 1

            x2 = routing_lat[point]
            y2 = routing_lon[point]
            dist_btn_node = distance(x1, y1, x2, y2) * 0.037 # 초(각도) >> km(거리)
            total_route_dist += dist_btn_node
            if point==4:
              break
        except:
          pass

        total_time = 60/23 * total_route_dist
        df2.loc[index,'총이동시간(분)'] = total_time
        df2.loc[index,'총경유거리(km)'] = total_route_dist

        df2.loc[index,'이동경로'] = ''
        df2.loc[index,'승하차순서'] = ''
        df2.at[index,'이동경로'] = routing_location
        df2.at[index,'승하차순서'] = routing_lat_lon

    # 조건을 만족하는 후보자가 없으면 거리를 늘린다.
    if len(df2) == 0:
        print('need to increase range_BY STEP4')
        pass

    #########################################################################
    ################################# STEP5 #################################

    ratio_index = {}
    len_df2 = len(df2)

    alpha_min = 0.10 # 동승객 최소 할인율
    beta_min = 0.10 # 동승객 최소 추가 할인율
    gamma = 0.05 # 혹시 최소 수익률
    # overlap_ratio = 0.4

    for _ in range(len_df2):
      for index, row in df2.iterrows():
        sort_last = []
        print(f'fellow : {index}')
        i_org = df2.loc[index,'원래시간(분)']
        i_rou = df2.loc[index,'경유시간(분)']
        a_org = df2.loc[index,'타깃원래시간(분)']
        a_rou = df2.loc[index,'타깃경유시간(분)']
        sort_last.append(max((i_rou-i_org)/i_org,(a_rou-a_org)/a_org))
        ratio_index[max((i_rou-i_org)/i_org,(a_rou-a_org)/a_org)] = index

        fellow_user = df2.loc[ratio_index[min(sort_last)]]['ID']
        fellow_index = ratio_index[min(sort_last)]

        a_ind_pred = int(4800 + (df2.loc[fellow_index,'타깃원래거리(km)']-1.6)*100/1.31 + (df2.loc[fellow_index,'타깃원래시간(분)']-96/23)*200)
        i_ind_pred = int(4800 + (df2.loc[fellow_index,'원래거리(km)']-1.6)*100/1.31 + (df2.loc[fellow_index,'원래시간(분)']-96/23)*200)
        driver_rou_pred = int(4800 + (df2.loc[fellow_index,'총경유거리(km)']-1.6)*100/1.31 + (df2.loc[fellow_index,'총이동시간(분)']-96/23)*200)

        det = a_ind_pred*(1-alpha_min-beta_min*(a_rou-a_org)/a_org)+ i_ind_pred*(1-alpha_min-beta_min*(i_rou-i_org)/i_org) - driver_rou_pred*(1+gamma)

        # if (a_ind_pred + i_ind_pred - driver_rou_pred)/driver_rou_pred > overlap_ratio:
        if (det > 0):
          print(f"최종적으로 'A'와 '{fellow_user}'을(를) 매칭해준다. ( {det:.2f} )")
          print()
          break
        else:
          df2 = df2.drop(index, axis=0)
          print(f"{fellow_user}, : {det:.2f} => 최종 매칭 X")
          print()
      # if (a_ind_pred + if_ind_pred - driver_rou_pred)/driver_rou_pred > overlap_ratio:
      if (det > 0):
        break

    # 조건을 만족하는 후보자가 없으면 거리를 늘린다.
    if len(df2) == 0:
        print('need to increase range_BY STEP5')
        pass

    break

df2

range : 0.1351km

fellow : 9
I, : -11996.38 => 최종 매칭 X

fellow : 2
B, : -10881.43 => 최종 매칭 X

fellow : 7
G, : -9955.97 => 최종 매칭 X

fellow : 37
최종적으로 'A'와 'KK'을(를) 매칭해준다. ( 14420.78 )



,ID,출발희망시각,최대대기시각,시간범위,출발위치,도착위치,타깃경유시간(분),타깃경유거리(km),타깃원래시간(분),타깃원래거리(km),경유시간(분),경유거리(km),원래시간(분),원래거리(km),총이동시간(분),총경유거리(km),이동경로,승하차순서
우선순위,,,,,,,,,,,,,,,,,,
37,KK,2023-03-10 17:30:00,2023-03-10 18:20:00,"(1050, 1100)","(300, 100)","(7.9506, 897.4383)",84.141840,32.254372,84.14184,32.254372,90.712549,34.773144,79.170824,30.348816,90.712549,34.773144,"[(200, 100), (200, 100), (40.7774, 957.0755), ...","[i_start, i_start, i_end, a_end]"
22,V,2023-03-10 17:35:00,2023-03-10 18:05:00,"(1055, 1085)","(100, 100)","(935.984, 870.9837)",189.685605,72.712815,84.14184,32.254372,102.880054,39.437354,102.880054,39.437354,189.685605,72.712815,"[(200, 100), (200, 100), (935.984, 870.9837), ...","[i_start, i_start, a_end, i_end]"
6,F,2023-03-10 17:40:00,2023-03-10 18:20:00,"(1060, 1100)","(200, 300)","(655.3076, 456.1976)",84.141840,32.254372,84.14184,32.254372,160.663921,61.587837,55.797726,21.389128,160.663921,61.587837,"[(200, 100), (200, 100), (40.7774, 957.0755), ...","[i_start, i_start, i_end, a_end]"
32,FF,2023-03-10 17:45:00,2023-03-10 18:35:00,"(1065, 1115)","(200, 200)","(79.5386, 905.0649)",84.141840,32.254372,84.14184,32.254372,90.402771,34.654395,78.571331,30.119010,90.402771,34.654395,"[(200, 100), (200, 100), (40.7774, 957.0755), ...","[i_start, i_start, i_end, a_end]"
31,EE,2023-03-10 18:10:00,2023-03-10 18:20:00,"(1090, 1100)","(100, 100)","(561.0248, 992.6755)",143.275002,54.922084,84.14184,32.254372,92.942389,35.627916,92.942389,35.627916,143.275002,54.922084,"[(200, 100), (200, 100), (561.0248, 992.6755),...","[i_start, i_start, a_end, i_end]"
29,CC,2023-03-10 18:10:00,2023-03-10 18:30:00,"(1090, 1110)","(300, 300)","(924.0397, 494.1039)",175.823301,67.398932,84.14184,32.254372,79.567605,30.500915,79.567605,30.500915,175.823301,67.398932,"[(200, 100), (200, 100), (924.0397, 494.1039),...","[i_start, i_start, a_end, i_end]"
39,MM,2023-03-10 18:10:00,2023-03-10 18:40:00,"(1090, 1120)","(200, 100)","(768.6046, 740.3399)",155.956792,59.783437,84.14184,32.254372,82.657014,31.685189,82.657014,31.685189,155.956792,59.783437,"[(200, 100), (200, 100), (768.6046, 740.3399),...","[i_start, i_start, a_end, i_end]"
36,JJ,2023-03-10 18:10:00,2023-03-10 19:00:00,"(1090, 1140)","(300, 300)","(476.4245, 363.4855)",84.141840,32.254372,84.14184,32.254372,155.210783,59.497467,36.860073,14.129695,155.210783,59.497467,"[(200, 100), (200, 100), (40.7774, 957.0755), ...","[i_start, i_start, i_end, a_end]"
42,PP,2023-03-10 18:20:00,2023-03-10 18:30:00,"(1100, 1110)","(200, 100)","(330.0045, 997.9304)",115.767316,44.377471,84.14184,32.254372,87.573476,33.569833,87.573476,33.569833,115.767316,44.377471,"[(200, 100), (200, 100), (330.0045, 997.9304),...","[i_start, i_start, a_end, i_end]"


In [22]:
C1 = a_ind_pred
C2 = (a_rou-a_org)/a_org
C3 = i_ind_pred
C4 = (i_rou-i_org)/i_org
C5 = driver_rou_pred
eplison = 0.03 # 실제 요금/예상요금 비율
gamma = 0.05 # 매칭당 혹시의 이익률

LB = 0.2
UB = 0.5

## matching ALGorithm
try:
  # Create a new model
  m = gp.Model('cost_optimization1')

  # Create variables
  alpha = m.addVar(vtype=GRB.CONTINUOUS, name='alpha')
  beta = m.addVar(vtype=GRB.CONTINUOUS, name='beta')

  # Set objective
  m.setObjective(C1*(1-(alpha+beta*C2))+C3*(1-(alpha+beta*C4))-(1+gamma)*C5, GRB.MINIMIZE)

  # Add constraint
  m.addConstr(max(C2,C4)*beta-UB*alpha <= 0,'const_1')
  m.addConstr(max(C2,C4)*beta-LB*alpha >= 0,'const_2')
  m.addConstr((C1*(1-(alpha+beta*C2))+C3*(1-(alpha+beta*C4))-(1+gamma)*C5) >= 0,'const_3')

  m.addConstr(alpha <= 0.5)
  m.addConstr(alpha >= 0)
  m.addConstr(beta <= 0.5)
  m.addConstr(beta >= 0)
  #m.addConstr(gamma <= 1)

  # Optimize model
  m.optimize()

  for v in m.getVars():
    print('%s : %g' % (v.varName, v.x))
    globals()[v.varName] = v.x
  print('Obj : %g' % m.objVal)

except gp.GurobiError as e:
  print('Error code ' + str(e.errno) + ': ' + str(e))

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 2 columns and 10 nonzeros
Model fingerprint: 0x1600fca4
Coefficient statistics:
  Matrix range     [1e-01, 5e+04]
  Objective range  [3e+03, 5e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-01, 2e+04]
Presolve removed 7 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.2044427e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.204442658e+03
alpha : 0.364456
beta : 0.5
Obj : 1204.44


In [23]:
print(f'타깃동승자   {alpha*100:.2f}+{beta*C2*100:.2f} => {(alpha+beta*C2)*100:.2f}% 할인')
print(f'동승자       {alpha*100:.2f}+{beta*C4*100:.2f}  => {(alpha+beta*C4)*100:.2f}% 할인')
print(f'타깃깃동승자 {C1}원 >> {(1-alpha+beta*C2)*C1:.0f}원')
print(f'동승자       {C3}원 >> {(1-alpha+beta*C4)*C3:.0f}원')
print(f'혹시가 얻는 최종 이익 >> {(1-alpha+beta*C2)*C1+(1-alpha+beta*C4)*C3-(1+eplison)*C5:.0f} 원')
print()
print(f'최적화 후 objective value: {(1-alpha-beta*C2)*C1+(1-alpha-beta*C4)*C3-(1+gamma)*C5:.0f} 원')

타깃동승자   36.45+0.00 => 36.45% 할인
동승자       36.45+7.29  => 43.73% 할인
타깃깃동승자 23133원 >> 14702원
동승자       21993원 >> 15581원
혹시가 얻는 최종 이익 >> 4903 원

최적화 후 objective value: 1204 원


In [25]:
#########################################################################
################################# STEP6 #################################
fellow_index = ratio_index[min(sort_last)]
fellow_info = df2.loc[fellow_index]
# df1에 매칭된 승객 제거
df1 = df1.drop(index=[target_index,fellow_index])

# df3에 추가
target_id = target_info.ID
fellow_id = fellow_info.ID
routing_point = [fellow_info.이동경로]
routing_order = [fellow_info.승하차순서]

df3 = pd.DataFrame(dict(zip(['타킷동승자(target)','매칭동승자(fellow)','이동경로','승하차순서'],[target_id, fellow_id, routing_point, routing_order])))
df3

,타킷동승자(target),매칭동승자(fellow),이동경로,승하차순서
0,A,KK,"[(200, 100), (200, 100), (40.7774, 957.0755), ...","[i_start, i_start, i_end, a_end]"
